<a href="https://colab.research.google.com/github/NakkaBhavith/AI/blob/main/Alzheimers(Randomforest).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

In [2]:
col_names = ['Subject ID', 'MRI ID', 'Group', 'Visit', 'MR Delay', 'M/F', 'Hand', 'Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF']
# load dataset
data = pd.read_csv("/content/drive/MyDrive/AI Data/oasis_longitudinal.csv")
data.head()

,Subject ID,MRI ID,Group,Visit,MR Delay,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034


In [3]:
data.drop(['Subject ID','MRI ID','Visit','MR Delay'], axis = 1, inplace = True)
# Checking for null values
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Group   373 non-null    object 
 1   M/F     373 non-null    object 
 2   Hand    373 non-null    object 
 3   Age     373 non-null    int64  
 4   EDUC    373 non-null    int64  
 5   SES     354 non-null    float64
 6   MMSE    371 non-null    float64
 7   CDR     373 non-null    float64
 8   eTIV    373 non-null    int64  
 9   nWBV    373 non-null    float64
 10  ASF     373 non-null    float64
dtypes: float64(5), int64(3), object(3)
memory usage: 32.2+ KB
None


In [4]:
pd.isnull(data).sum() 

Group     0
M/F       0
Hand      0
Age       0
EDUC      0
SES      19
MMSE      2
CDR       0
eTIV      0
nWBV      0
ASF       0
dtype: int64

In [5]:
data = data.dropna(axis=0, how='any')
pd.isnull(data).sum()

Group    0
M/F      0
Hand     0
Age      0
EDUC     0
SES      0
MMSE     0
CDR      0
eTIV     0
nWBV     0
ASF      0
dtype: int64

In [6]:
# Converting the categorical variable into numerical
varlist1 =  ['M/F']
varlist2 = ['Group']
varlist3 = ['Hand']

# Defining the map function
def binary_map1(x):
    return x.map({'M': 1, "F": 0})
def binary_map2(y):
    return y.map({'Demented' : 1, 'Nondemented' : 0, 'Converted' : 1})
def binary_map3(z):
    return z.map({'R' : 1, 'L' : 0})

data[varlist1] = data[varlist1].apply(binary_map1)
data[varlist2] = data[varlist2].apply(binary_map2)
data[varlist3] = data[varlist3].apply(binary_map3)

data

,Group,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,0,1,1,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,0,1,1,88,14,2.0,30.0,0.0,2004,0.681,0.876
5,0,0,1,88,18,3.0,28.0,0.0,1215,0.710,1.444
6,0,0,1,90,18,3.0,27.0,0.0,1200,0.718,1.462
7,0,1,1,80,12,4.0,28.0,0.0,1689,0.712,1.039
...,...,...,...,...,...,...,...,...,...,...,...
368,1,1,1,82,16,1.0,28.0,0.5,1693,0.694,1.037
369,1,1,1,86,16,1.0,26.0,0.5,1688,0.675,1.040
370,0,0,1,61,13,2.0,30.0,0.0,1319,0.801,1.331
371,0,0,1,63,13,2.0,30.0,0.0,1327,0.796,1.323


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Applying scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['Age', 'EDUC', 'SES', 'MMSE', 'eTIV', 'ASF']
data[num_vars] = scaler.fit_transform(data[num_vars])

data

,Group,M/F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,0,1,1,0.710526,0.470588,0.25,0.884615,0.0,0.981069,0.696,0.009845
1,0,1,1,0.736842,0.470588,0.25,1.000000,0.0,1.000000,0.681,0.000000
5,0,0,1,0.736842,0.705882,0.50,0.923077,0.0,0.121381,0.710,0.798875
6,0,0,1,0.789474,0.705882,0.50,0.884615,0.0,0.104677,0.718,0.824191
7,0,1,1,0.526316,0.352941,0.75,0.923077,0.0,0.649220,0.712,0.229255
...,...,...,...,...,...,...,...,...,...,...,...
368,1,1,1,0.578947,0.588235,0.00,0.923077,0.5,0.653675,0.694,0.226442
369,1,1,1,0.684211,0.588235,0.00,0.846154,0.5,0.648107,0.675,0.230661
370,0,0,1,0.026316,0.411765,0.25,1.000000,0.0,0.237194,0.801,0.639944
371,0,0,1,0.078947,0.411765,0.25,1.000000,0.0,0.246102,0.796,0.628692


In [8]:
#split dataset in features and target variable
feature_cols = ['M/F','Hand','Age', 'EDUC', 'SES', 'MMSE', 'CDR', 'eTIV', 'nWBV', 'ASF']
X = data[feature_cols] # Features
y = data.Group # Target variable

In [9]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [10]:
# Create Random Forest classifer object
clf = RandomForestClassifier()

# Train Random Forest Classifer
clf = clf.fit(X_train,y_train)

In [11]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_pred1 = clf.predict(X_train)

In [12]:
print("Testing Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Training Accuracy:",metrics.accuracy_score(y_train, y_pred1))

Testing Accuracy: 0.9436619718309859
Training Accuracy: 1.0


In [16]:
# Create Random Forest classifer object
clf = RandomForestClassifier(criterion="entropy", max_depth=7)

# Train Random Forest Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)
y_pred1 = clf.predict(X_train)

# Model Accuracy, how often is the classifier correct?
print("Testing Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Training Accuracy:",metrics.accuracy_score(y_train, y_pred1))

Testing Accuracy: 0.9436619718309859
Training Accuracy: 0.9811320754716981


In [17]:
confusion_matrix(y_test, y_pred)

array([[71,  1],
       [ 7, 63]])

In [18]:
confusion_matrix(y_train, y_pred1)

array([[118,   0],
       [  4,  90]])

In [19]:
import pickle
# Save the model
filename = 'model.pkl'
pickle.dump(clf, open(filename, 'wb'))

In [20]:
list_of_columns = data.columns
input_data=pd.DataFrame(columns=list_of_columns)
input_data.drop(['Group'], axis='columns', inplace=True)

input_data.at[0, 'M/F'] = input('enter Gender : ')
input_data.at[0, 'Hand'] = input('enter Hand : ')
input_data.at[0, 'Age'] = int(input('enter Age : '))
input_data.at[0, 'EDUC'] = int(input('enter education : '))
input_data.at[0, 'SES'] = float(input('enter SES : '))
input_data.at[0, 'MMSE'] = float(input('enter MMSE : '))
input_data.at[0, 'CDR'] = float(input('enter CDR : '))
input_data.at[0, 'eTIV'] = float(input('enter eTIV : '))
input_data.at[0, 'nWBV'] = float(input('enter nWBV : '))
input_data.at[0, 'ASF'] = float(input('enter ASF : '))

# Defining the map function
def binary_map1(x):
    return x.map({'M': 1, "F": 0})
def binary_map3(z):
    return z.map({'R': 1, "L": 0})

input_data[varlist1] = input_data[varlist1].apply(binary_map1)
input_data[varlist3] = input_data[varlist3].apply(binary_map3)

model = pickle.load(open('model.pkl', 'rb'))
prediction = model.predict(input_data)
result = prediction[0]
if(result == 1):
  print('You Will suffer with Alzheimers')
if(result == 0):
  print('You will not suffer with Alzheimers')

enter Gender : M
enter Hand : R
enter Age : 90
enter education : 12
enter SES : 3
enter MMSE : 27
enter CDR : 1
enter eTIV : 1800
enter nWBV : 0.9
enter ASF : 1
You Will suffer with Alzheimers
